## HW 3
Botasheva Zhanna

## Что в векторе твоем?

### Цель:

В этом ДЗ вы освоите работу с предобученными векторными представлениями.
В качестве данных возьмите либо датасет, собранный в первом занятии (предпочтительно), либо скачайте данные с отзывами на фильмы с сайта IMDB (https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews), в которых для каждого отзыва поставлена семантическая оценка - "позитивный" или "негативный".

- Разбейте собранные данные на train/test, отложив 20-30% наблюдений для тестирования.
Примените tf-idf преобразование для текстового описания. Используйте как отдельные токены, так и биграммы, отсейте стоп-слова, а также слова, которые встречаются слишком редко или слишком часто (параметры min/max_df), не забудьте убрать l2 регуляризацию, которая по умолчанию включена.
- Обучите random forest или градиентный бустинг (LightGBM или catboost) на полученных векторах и подберите оптимальную комбинацию гиперпараметров с помощью GridSearch
- Теперь воспользуйтесь предобученными word2vec/fasttext эмбеддингами для векторизации текста. Векторизуйте тексты с помощью метода word2vec/fasttext c весами tf-idf
- Совет: для текстов на русском языке можно взять предобученные эмбеддинги с сайта rusvectores https://rusvectores.org/ru/models/ (вам подходят эмбеддинги с параметром тэгсет НЕТ). Для английского языка можете воспользоваться word2vec, обученными на Google News
Повторите эксперимент из пункта 4 с использованием полученных в пункте 5 векторов

In [19]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
from sklearn.metrics import *
import warnings
warnings.filterwarnings("ignore")

In [20]:
import nltk   # Natural Language Toolkit
from nltk.corpus import stopwords
from string import punctuation
from pymystem3 import Mystem

In [140]:
df = pd.read_csv('book.csv')

In [141]:
df.head()

,Unnamed: 0,Название книги,Автор,Рейтинг,ID,Год,Кол-во страниц,Цена,Описание,0
0,0,Девушка Online,Сагг Зои,2.0,7467401.0,2020.0,352.0,375.0,"Зои Сагг, известная как Zoella, - двадцатичеты...",NaN
1,1,Мятная сказка. Специальное издание,Полярный Александр,2.0,7740488.0,2019.0,160.0,690.0,"«Мятная сказка» - культовая книга, принесшая с...",NaN
2,2,Реннвинд. Сердце тьмы,Стенберг Леа,4.3,7965332.0,2023.0,384.0,440.0,"Когда кажется, что все секреты раскрыты, стано...",NaN
3,3,"Трансформатор 3. В чем сила, бро?",Портнягин Дмитрий,1.5,7769954.0,2019.0,304.0,125.0,Дмитрий Портнягин - известный российский предп...,NaN
4,4,Все это время,Дотри Микки,NaN,7820141.0,2020.0,384.0,360.0,"Рейчел Липпинкотт, Микки Дотри — авторы, котор...",NaN


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      473 non-null    int64  
 1   Название книги  471 non-null    object 
 2   Автор           471 non-null    object 
 3   Рейтинг         233 non-null    float64
 4   ID              471 non-null    float64
 5   Год             471 non-null    float64
 6   Кол-во страниц  469 non-null    float64
 7   Цена            466 non-null    float64
 8   Описание        471 non-null    object 
 9   0               2 non-null      float64
dtypes: float64(6), int64(1), object(3)
memory usage: 37.1+ KB


Избавимся от Nan в Рейтинге

In [143]:
#df['Рейтинг'] = df['Рейтинг'].fillna(0) 
df = df.dropna(subset=['Рейтинг'])

Удалим лишние столбцы

In [144]:
df = df.drop(columns=['Unnamed: 0', '0'])

In [145]:
df.head()

,Название книги,Автор,Рейтинг,ID,Год,Кол-во страниц,Цена,Описание
0,Девушка Online,Сагг Зои,2.0,7467401.0,2020.0,352.0,375.0,"Зои Сагг, известная как Zoella, - двадцатичеты..."
1,Мятная сказка. Специальное издание,Полярный Александр,2.0,7740488.0,2019.0,160.0,690.0,"«Мятная сказка» - культовая книга, принесшая с..."
2,Реннвинд. Сердце тьмы,Стенберг Леа,4.3,7965332.0,2023.0,384.0,440.0,"Когда кажется, что все секреты раскрыты, стано..."
3,"Трансформатор 3. В чем сила, бро?",Портнягин Дмитрий,1.5,7769954.0,2019.0,304.0,125.0,Дмитрий Портнягин - известный российский предп...
5,Злая лисица,Чо Кэт,4.0,7841391.0,2022.0,480.0,460.0,ЛУЧШЕЕ YOUNG ADULT ФЭНТЕЗИ 2021 ГОДА! ВПЕРВЫЕ ...


Для упрощения задачи добавим столбец "Рекомендация", чтобы использовать его в качестве целевой перемнной.

In [146]:
df['Рекомендация'] = df['Рейтинг'].apply(lambda x: 'Рекомендуют' if x >= 4 else 'Не рекомендуют')
df.head()

,Название книги,Автор,Рейтинг,ID,Год,Кол-во страниц,Цена,Описание,Рекомендация
0,Девушка Online,Сагг Зои,2.0,7467401.0,2020.0,352.0,375.0,"Зои Сагг, известная как Zoella, - двадцатичеты...",Не рекомендуют
1,Мятная сказка. Специальное издание,Полярный Александр,2.0,7740488.0,2019.0,160.0,690.0,"«Мятная сказка» - культовая книга, принесшая с...",Не рекомендуют
2,Реннвинд. Сердце тьмы,Стенберг Леа,4.3,7965332.0,2023.0,384.0,440.0,"Когда кажется, что все секреты раскрыты, стано...",Рекомендуют
3,"Трансформатор 3. В чем сила, бро?",Портнягин Дмитрий,1.5,7769954.0,2019.0,304.0,125.0,Дмитрий Портнягин - известный российский предп...,Не рекомендуют
5,Злая лисица,Чо Кэт,4.0,7841391.0,2022.0,480.0,460.0,ЛУЧШЕЕ YOUNG ADULT ФЭНТЕЗИ 2021 ГОДА! ВПЕРВЫЕ ...,Рекомендуют


In [147]:
df['Рекомендация'].value_counts(dropna=False)

Рекомендация
Рекомендуют       119
Не рекомендуют    114
Name: count, dtype: int64

Цеоевая переменная сбалансирована, возможно стоило спарсить больше данных, попробуем поработать с тем что есть.

Разобьем выборку на две части: тренировочную (train) и тестовую выборку (test). На test будем проверять обобщающая способность модели

In [148]:
from sklearn.model_selection import train_test_split as tts

X_train,X_test = tts(data_preprocessed,test_size=0.3,random_state=42)

In [149]:
# Тренировачная выборка

corpus_tr = list(X_train['Описание'])  # корпус 
target_tr = list(X_train['Рекомендация'])   # целевая функция

In [150]:
# Тестовая выборка

corpus_te = list(X_test['Описание'])  # корпус 
target_te = list(X_test['Рекомендация'])   # целевая функция

### Варианты TF-IDF
Все варианты преобразования документов в корпусе используя TF-IDF для создания векторного представления слов перечислены ниже

In [151]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
russian_stopwords = stopwords.words("russian")

# минимальная частота слов
v1_vectoriser = TfidfVectorizer(min_df=3,norm=None)
v1_vectoriser.fit(corpus_tr)
X1 = v1_vectoriser.transform(corpus_tr)

v2_vectoriser = TfidfVectorizer(min_df=2,norm=None)
v2_vectoriser.fit(corpus_tr)
X2 = v2_vectoriser.transform(corpus_tr)

v3_vectoriser = TfidfVectorizer(min_df=1,norm=None)
v3_vectoriser.fit(corpus_tr)
X3 = v3_vectoriser.transform(corpus_tr)

In [152]:
# стоп слова варианты
v4_vectoriser = TfidfVectorizer(min_df=3,norm=None,stop_words=russian_stopwords)
v4_vectoriser.fit(corpus_tr)
X4 = v4_vectoriser.transform(corpus_tr)

v5_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=russian_stopwords)
v5_vectoriser.fit(corpus_tr)
X5 = v5_vectoriser.transform(corpus_tr)

v6_vectoriser = TfidfVectorizer(min_df=1,norm=None,stop_words=russian_stopwords)
v6_vectoriser.fit(corpus_tr)
X6 = v6_vectoriser.transform(corpus_tr)

In [153]:
# н-грам варианты
v7_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=russian_stopwords,ngram_range=(1,1))
v7_vectoriser.fit(corpus_tr)
X7 = v7_vectoriser.transform(corpus_tr)

v8_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=russian_stopwords,ngram_range=(1,2))
v8_vectoriser.fit(corpus_tr)
X8 = v8_vectoriser.transform(corpus_tr)

v9_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=russian_stopwords,ngram_range=(1,3))
v9_vectoriser.fit(corpus_tr)
X9 = v9_vectoriser.transform(corpus_tr)

v10_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=russian_stopwords,ngram_range=(2,2))
v10_vectoriser.fit(corpus_tr)
X10 = v10_vectoriser.transform(corpus_tr)

v11_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=russian_stopwords,ngram_range=(3,3))
v11_vectoriser.fit(corpus_tr)
X11 = v11_vectoriser.transform(corpus_tr)

v12_vectoriser = TfidfVectorizer(min_df=2,norm=None,stop_words=russian_stopwords,ngram_range=(4,4))
v12_vectoriser.fit(corpus_tr)
X12 = v12_vectoriser.transform(corpus_tr)

In [154]:
# максимальная частота слов
v13_vectoriser = TfidfVectorizer(min_df=2,max_df=0.9,norm=None,stop_words=russian_stopwords)
v13_vectoriser.fit(corpus_tr)
X13 = v13_vectoriser.transform(corpus_tr)

v14_vectoriser = TfidfVectorizer(min_df=2,max_df=0.8,norm=None,stop_words=russian_stopwords)
v14_vectoriser.fit(corpus_tr)
X14 = v14_vectoriser.transform(corpus_tr)

v15_vectoriser = TfidfVectorizer(min_df=2,max_df=0.7,norm=None,stop_words=russian_stopwords)
v15_vectoriser.fit(corpus_tr)
X15 = v15_vectoriser.transform(corpus_tr)

### Варианты Моделей

Начнем с базового RandomForest (не очень глубокого) <code>model_srf</code>, а потом попробуем улучшить результат с помощью <code>model_drf</code> и <code>model_ocb</code>

In [155]:
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

model_srf = RandomForestClassifier(n_estimators=10,random_state=32)  # не глубокий случайный лес
model_drf = RandomForestClassifier(n_estimators=40,random_state=32)  # глубокий случайный лес
model_ocb = CatBoostClassifier(silent=True)  # градиентный бустинг

In [156]:
from sklearn.metrics import accuracy_score

# вспомогательная функция для оценки модели

def evaluate_tfidf(X,vect,name,model):

    print(f'case_id: :{name}')
    print('==================================')
    
    # train model
    model.fit(X,target_tr)
    y_model = model.predict(X)
    print(f'train: {accuracy_score(target_tr,y_model)}')
    
    X = vect.transform(corpus_te)
    y_model = model.predict(X)
    print(f'test: {accuracy_score(y_model,target_te)}')
    
    print('==================================','\n')

### Минимальная частота слов

Проверим как влияет фильтр минимальной частоты слов (<code>min_df</code>) в документе на результат 
- <code>v1</code> (<code>min_df=3</code> фильтруем слова которые появляются в документов меньше 3 раз)
- <code>v2</code> (<code>min_df=2</code> фильтруем слова которые появляются в документов меньше 2 раз)
- <code>v3</code> (<code>min_df=1</code> фильтруем слова которые появляются в документов меньше 1 раза)

Как мы видим, эффект от фильтрации часто встречающихся слов разный

In [157]:
evaluate_tfidf(X3,v3_vectoriser,'v3',model_srf)
evaluate_tfidf(X2,v2_vectoriser,'v2',model_srf)
evaluate_tfidf(X1,v1_vectoriser,'v1',model_srf) 

case_id: :v3
train: 0.9546827794561934
test: 0.6126760563380281

case_id: :v2
train: 0.972809667673716
test: 0.5352112676056338

case_id: :v1
train: 0.9758308157099698
test: 0.5985915492957746



### Стоп Слова

- Попробуем повысить точность с помощью фильтрации не релевантных слов (стоп слова) 
- При использовании разных настроек <code>min_df</code> точность модели меняется по разному; <code>min_df=3</code> дает наиболее хороший результат

Вариатны:
- <code>v4</code> (фильтрация стоп слов с <code>min_df=3</code>)
- <code>v5</code> (фильтрация стоп слов с <code>min_df=2</code>)
- <code>v6</code> (фильтрация стоп слов с <code>min_df=1</code>)

In [158]:
evaluate_tfidf(X4,v4_vectoriser,'v6',model_srf)
evaluate_tfidf(X5,v5_vectoriser,'v5',model_srf)
evaluate_tfidf(X6,v6_vectoriser,'v4',model_srf) 

case_id: :v6
train: 0.9637462235649547
test: 0.5492957746478874

case_id: :v5
train: 0.9667673716012085
test: 0.5774647887323944

case_id: :v4
train: 0.9697885196374623
test: 0.6267605633802817



###  н-граммы

- Вариант <code>v4</code> дает нам точность 0.63 на тестовой выборке, используем этот вариант и проверим разные комбинации н-грамм
- Как мы видим, улучшении никаких нет на тестовой выборке, при использовании bigram,trigram, точность сильно снизилось,  базовые токены (unigram) являются критичным для высокой точности модели

Варианты:

- <code>v7</code> ngram_range=(1,1) (Только униграммы)
- <code>v8</code> ngram_range=(1,2) (Униграммы и биграммы)
- <code>v9</code> ngram_range=(1,3) (униграммы, биграммы и триграммы)

In [159]:
evaluate_tfidf(X9,v9_vectoriser,'v9',model_srf)
evaluate_tfidf(X8,v8_vectoriser,'v8',model_srf)
evaluate_tfidf(X7,v7_vectoriser,'v7',model_srf) 

case_id: :v9
train: 0.9697885196374623
test: 0.6056338028169014

case_id: :v8
train: 0.9637462235649547
test: 0.6126760563380281

case_id: :v7
train: 0.9667673716012085
test: 0.5774647887323944



Проверим еще варианты:

- <code>v10</code> ngram_range=(1,1) (Только униграммы)
- <code>v11</code> ngram_range=(2,2) (Только биграммы)
- <code>v12</code> ngram_range=(3,3) (Только триграммы)

In [160]:
evaluate_tfidf(X12,v12_vectoriser,'v12',model_srf)
evaluate_tfidf(X11,v11_vectoriser,'v11',model_srf)
evaluate_tfidf(X10,v10_vectoriser,'v10',model_srf) 

case_id: :v12
train: 0.6525679758308157
test: 0.6267605633802817

case_id: :v11
train: 0.7250755287009063
test: 0.6197183098591549

case_id: :v10
train: 0.9788519637462235
test: 0.6197183098591549



### Максимальная частота

В задании так же упоминается максимальная частота слов <code>max_df</code> в векторизаторе, попробуем варианты:
- <code>v13</code> <code>max_df=0.9</code> (фильтруем слова которые появляются в 90% документов)
- <code>v14</code> <code>max_df=0.8</code> (фильтруем слова которые появляются в 80% документов)
- <code>v15</code> <code>max_df=0.7</code> (фильтруем слова которые появляются в 70% документов)

Как мы видим, улучшении никаких нет, при уменьшении часто встречаюшихся слов, точность падает

In [161]:
evaluate_tfidf(X13,v13_vectoriser,'v13',model_srf)
evaluate_tfidf(X14,v14_vectoriser,'v14',model_srf)
evaluate_tfidf(X15,v15_vectoriser,'v15',model_srf)

case_id: :v13
train: 0.9667673716012085
test: 0.5774647887323944

case_id: :v14
train: 0.9667673716012085
test: 0.5774647887323944

case_id: :v15
train: 0.9667673716012085
test: 0.5774647887323944



### Выбор модели

- Базовая модель случайного леса не очень глубокая, соответственно может не очень хорошо определить закономерности в данных, попробуем другие варианты перечислены в начале раздела
- Воспользуемся вариантом который показал наилучшую обобщающию способность на тестовой выборке (<code>v4</code>)
- CatBoost хорошо оптимизируется под любые табличные данные, и как мы видим улучшения нет, обе модели показывают примерно идентичный результат
- Соответственно возможно что улучшении на тестовой выборке с TF-IDF (помимо токенизации) особо нет, и нужно проверить эмбеддинговый подход 

In [162]:
# 1775 признаков

X4.shape

(331, 1775)

In [163]:
evaluate_tfidf(X4,v4_vectoriser,'v4_deeprf',model_drf)
evaluate_tfidf(X4,v4_vectoriser,'v4_catboost',model_ocb)

case_id: :v4_deeprf
train: 0.9939577039274925
test: 0.5915492957746479

case_id: :v4_catboost
train: 0.918429003021148
test: 0.5915492957746479



### TF-IDF GridSearchCV

- В ручную подборка гиперпараметров может оказаться не самым оптимальным вариантов, поэтому воспользуемся <code>GridSearchCV</code> методом кросс-валидации для подборки гиперпараметров модели
- Какую конкретные параметры нужно оптимизировать не уточняется, попробуем найти более оптимальные параметры для модели случайного леса, так как CatBoost показал хуже результат

У случайного леса много параметров которые можно перепобовать, ограничимся:
- <code>max_depth</code> максимальная глубина разбиения 
- <code>n_estimators</code> количество решаущих деревьев
- <code>criterion</code> критерии оценки разбиения в узле
- <code>min_samples_leaf</code> минимальное количество образцов, необходимое для нахождения в листовом узле каждого дерева
- <code>min_samples_split</code> Минимальное количество образцов, необходимое для разбиения внутреннего узла каждого дерева

Из соображении времени, количество построенных деревьев ограничивается (e_stimators=100)

In [164]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
import numpy as np


def evaluate_cv(X,name,base_estimator):

    print(f'case_id: :{name}')
    print('==================================')
    
    rs_space={'max_depth':list(np.arange(10,60,10)) + [None],
              'n_estimators':np.arange(40,120, step=20),
              'criterion':['gini','entropy'],
              'min_samples_leaf':[1,2,3,4],
               'min_samples_split':np.arange(2,6, step=2),
              }

    model = GridSearchCV(base_estimator,
                         rs_space, 
                         scoring='accuracy', 
                         n_jobs=-1, 
                         cv=3)
    
    # train model
    model.fit(X,target_tr)
    
    print('Best hyperparameters are: '+str(model.best_params_))
    print('Best score is: '+str(model.best_score_))
    
    print('==================================','\n')
    


Кросс валидация нам дает немного хуже результат чем на train/test разбиение; в итоге мы получаем **точность 0.63** на тестовой выборке в cv 

In [165]:
%%time

model_drf = RandomForestClassifier(random_state=32)
evaluate_cv(X4,'random_forest_cv',model_srf)

case_id: :random_forest_cv
Best hyperparameters are: {'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 60}
Best score is: 0.6345345345345346

CPU times: total: 3.81 s
Wall time: 1min 59s


Проверим точность на на всей тестовой и отложенной тестовой выборке на больше объеме данных у нас модель показывает хуже результат

In [166]:
best_model = RandomForestClassifier(**{'criterion': 'entropy', 
                                     'max_depth': 40, 
                                     'min_samples_leaf': 3, 
                                     'min_samples_split': 2, 
                                     'n_estimators': 100, 
                                     'random_state':32})

evaluate_tfidf(X4,v4_vectoriser,'v4_rf_best',best_model)

case_id: :v4_rf_best
train: 0.972809667673716
test: 0.5915492957746479



### Эмбеддинги

Для нашей задачи возьмем предобученные эмбеддинги (варианты без тагсет)
- **geowac_lemmas_none_fasttextskipgram_300_5_2020**

In [167]:
# Не забываем про веса tfidf; создадим словарь
tfidf_weights = dict(zip(list(v4_vectoriser.vocabulary_.keys()),v4_vectoriser.idf_))

In [168]:
import gensim
import urllib.request
import zipfile

# название и URL
we_models = {"geowac_lemmas_none_fasttextskipgram_300_5_2020": "http://vectors.nlpl.eu/repository/20/213.zip",}

In [173]:
# сохраняем модель
def get_models(model_url, model_name, path_to_save="C:\\Users\\Zhanna Botasheva\\Desktop\\"):
    model_path = path_to_save + model_name + ".zip"
    urllib.request.urlretrieve(model_url, model_path)

for model_name, model_url in we_models.items():
    get_models(model_url, model_name)

In [174]:
# Функция для чтения word2vec / FastText

def open_model(model_name,model_path, is_fasttext = True):
    
    # word2vec (model.bin)
    if is_fasttext == False:
        model_file = model_path + model_name + ".zip"
        with zipfile.ZipFile(model_file, 'r') as archive:
            stream = archive.open('model.bin')
            model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)
            
    # fasttext (model.model)
    else:
        model_file = model_path + model_name
        model = gensim.models.KeyedVectors.load(model_file + "/model.model")
    return model

Распакуем эмбеддинги и загружаем вектора

In [175]:
with zipfile.ZipFile("C:\\Users\\Zhanna Botasheva\\Desktop\\geowac_lemmas_none_fasttextskipgram_300_5_2020.zip", 'r') as zip_ref: 
    zip_ref.extractall("C:\\Users\\Zhanna Botasheva\\Desktop\\geowac_lemmas_none_fasttextskipgram_300_5_2020")

In [177]:
# загружаем KeyedVectors эмбеддинговый вектора 
geowac_model = open_model('geowac_lemmas_none_fasttextskipgram_300_5_2020','C:\\Users\\Zhanna Botasheva\\Desktop\\')

### Токенизация документов

- Токенизируем все документы в тренировочной выборке/копрусе, будем использовать стандартный метод из nltk <code>word_tokenize</code>
- Токенизированные документы сохраним в <code>lst_corpus_tr</code> и <code>lst_corpus_te</code> для <code>train</code> и <code>test</code> выборки

In [180]:
from nltk.tokenize import word_tokenize

# Preprocessing, returns list instead
def tokenise_for_word2vec(text):
    
    text = text.lower() #changes to lower case
    tokens = word_tokenize(text) #tokenize the text
    
    clean_list = [] 
    for token in tokens:
        clean_list.append(token)
        
    return clean_list

In [182]:
import nltk

# Загрузка необходимых ресурсов
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Zhanna
[nltk_data]     Botasheva\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [183]:
lst_corpus_tr = []
for doc in corpus_tr:
    lst_corpus_tr.append(tokenise_for_word2vec(doc))
    
lst_corpus_te = []
for doc in corpus_te:
    lst_corpus_te.append(tokenise_for_word2vec(doc))

### Усредненные Вектора (без TF-IDF весов)

- Воспользуемся самым простым подходом; для каждого документа мы возьмем **усредненный эмбеддинговый вектор** всех слов которые в него входят
- Размерность данных с эмбеддингами получается намного ниже (300) чем в TF-IDF (1775) что должно сказаться на скорости обучения модели
- Сохраняем эмбеддинг для каждого документа в <code>X_tr</code> и <code>X_te</code> для <code>train</code> и <code>test</code> выборки 

In [195]:
# Get average embedding vector for each text

def doc_vectoriser(doc, model):
    
    doc_vector = []
    num_words = 0
    
    for word in doc:
        try:
            if num_words == 0:
                doc_vector = model[word]
            else:
                doc_vector = np.add(doc_vector, model[word])
            num_words += 1
        except:
            pass  # if embedding vector isn't found
     
    return np.asarray(doc_vector) / num_words


def doc_vectoriser(doc, model):
    doc_vector = np.zeros((300,))  # Предполагаем, что размерность эмбеддингов 300
    num_words = 0
    
    for word in doc:
        if word in model:
            doc_vector = np.add(doc_vector, model[word])
            num_words += 1
            
    # Избегаем деления на ноль
    return doc_vector / num_words if num_words > 0 else doc_vector

X_tr = np.array([doc_vectoriser(doc, geowac_model) for doc in lst_corpus_tr])
X_te = np.array([doc_vectoriser(doc, geowac_model) for doc in lst_corpus_te])

In [196]:
X_tr = []
for doc in lst_corpus_tr:
    X_tr.append(doc_vectoriser(doc,geowac_model))
    

X_te = []
for doc in lst_corpus_te:
    X_te.append(doc_vectoriser(doc,geowac_model))

### Усредненные Вектора (c TF-IDF весами)

- Мы сохранили веса значимости слов (токенов) в документах корпуса в <code>tfidf_weights</code>
- Сохраняем взвешенные эмбеддинг для каждого документа в <code>Xw_tr</code> и <code>Xw_te</code> для <code>train</code> и <code>test</code> выборки 

In [197]:
# Get average embedding vector for each text 

def doc_vectoriser_tfidf(doc, model):
    doc_vector = np.zeros((300,))  # Предполагаем, что размерность эмбеддингов 300
    num_words = 0
    
    for word in doc:
        if word in model:
            if word in tfidf_weights:
                doc_vector = np.add(doc_vector, model[word] * tfidf_weights[word])
            else:
                doc_vector = np.add(doc_vector, model[word])
            num_words += 1
            
    return doc_vector / num_words if num_words > 0 else doc_vector

Xw_tr = np.array([doc_vectoriser_tfidf(doc, geowac_model) for doc in lst_corpus_tr])
Xw_te = np.array([doc_vectoriser_tfidf(doc, geowac_model) for doc in lst_corpus_te])

In [198]:
Xw_tr = []
for doc in lst_corpus_tr:
    Xw_tr.append(doc_vectoriser_tfidf(doc,geowac_model))
    
Xw_te = []
for doc in lst_corpus_te:
    Xw_te.append(doc_vectoriser_tfidf(doc,geowac_model))

###  Варианты Моделей

Повторим то что мы делали в части I:

- В плане моделей, попробуем два подхода ансаблья, случайный лес (который использует бэггинт и метод случайного пространства) и градиентный бустинг СatBoost 
- Начнем с базового RandomForest (не очень глубокого) <code>model_srf</code>, а потом попробуем улучшить результат с помошью <code>model_drf</code> и <code>model_ocb</code>

In [199]:
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

model_srf = RandomForestClassifier(n_estimators=10,random_state=32)
model_drf = RandomForestClassifier(n_estimators=40,random_state=32)
model_ocb = CatBoostClassifier(silent=True)

In [200]:
from sklearn.metrics import accuracy_score

# Импортируем матричу фич (эмбеддинги) X_tr, X_te
# target_tr, target_te глоб 

def evaluate_embedding(X_tr, X_te, name, model):
    print(f'case_id: :{name}')
    print('==================================')
    
    try:
        # Train model
        model.fit(X_tr, target_tr)
        y_model_train = model.predict(X_tr)
        print(f'train: {accuracy_score(target_tr, y_model_train)}')
        
        y_model_test = model.predict(X_te)
        print(f'test: {accuracy_score(y_model_test, target_te)}')
    except Exception as e:
        print(f'Error during training or evaluation: {e}')
        
    print('==================================\n')


### Варианты Моделей (c TF-IDF весами)

- Модели **с TF-IDF весами** для эмбеддингов
- Точность с весами не улучшилась

In [201]:
evaluate_embedding(X_tr,X_te,'geowac_rf',model_drf)

case_id: :geowac_rf
train: 0.9939577039274925
test: 0.5774647887323944



In [202]:
evaluate_embedding(X_tr,X_te,'geowac_cat',model_ocb)

case_id: :geowac_cat
train: 0.9939577039274925
test: 0.6267605633802817



In [203]:
evaluate_embedding(Xw_tr,Xw_te,'geowac_rf_tfidf_weights',model_drf)

case_id: :geowac_rf_tfidf_weights
train: 0.9939577039274925
test: 0.6338028169014085



In [204]:
evaluate_embedding(Xw_tr,Xw_te,'geowac_cat_tfidf_weights',model_ocb)

case_id: :geowac_cat_tfidf_weights
train: 0.9939577039274925
test: 0.6197183098591549



### GridSearchCV (без TF-IDF весов)

- Воспользуемся той же функции что и раньше для того чтобы оптимизировать гиперпараметры модели
- Будем использовать случайны лес, и найдем наилучшие гиперпараметры используя кросс-валидацию <code>GridSearchCV</code>

In [205]:
%%time

model_drf = RandomForestClassifier(random_state=32)
evaluate_cv(np.array(X_tr),'geowac_rf_cv',model_drf)

case_id: :geowac_rf_cv
Best hyperparameters are: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 40}
Best score is: 0.6343707343707344

CPU times: total: 1.72 s
Wall time: 1min 20s


In [207]:
best_model_rf = RandomForestClassifier(**{'criterion': 'entropy', 
                                       'max_depth': 10, 
                                       'min_samples_leaf': 1, 
                                       'min_samples_split': 4, 
                                       'n_estimators': 100,
                                       'random_state':32})

In [208]:
evaluate_embedding(X_tr,X_te,'geowac_rf_tfidf_weights',best_model_rf)

case_id: :geowac_rf_tfidf_weights
train: 0.9939577039274925
test: 0.6056338028169014



### GridSearchCV (c TF-IDF весами)

- Повторяем оптимизацию гиперпараметров для эмбеддингов с весами TF-IDF
- Умножив веса из TF-IDF на эмбеддинги модель не много улучшилась

In [209]:
%%time

model_drf = RandomForestClassifier(random_state=32)
evaluate_cv(np.array(Xw_tr),'geowac_rf_cv_tfidf_weights',model_drf)

case_id: :geowac_rf_cv_tfidf_weights
Best hyperparameters are: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 80}
Best score is: 0.6526344526344526

CPU times: total: 1.95 s
Wall time: 1min 16s


Использование TF-IDF с Random Forest действительно показывает небольшое улучшение как в точности, так и в устойчивости модели. Это может быть связано с тем, что TF-IDF фокусирует внимание на более значимых словах для каждого документа, придавая им больший вес в расчёте вектора. Хотя точность модели увеличивается при использовании TF-IDF, улучшение незначительное, что может указывать на то, что вектора эмбеддингов уже улавливают важные характеристики текста.